# Example: SCSCP client in Python3 calculates Groebner basis with Singular

SymPy (http://www.sympy.org/) operates with multivariate polynomials and implements Groebner basis calculation. In this example we demonstrate the alternative of computing Groebner basis connecting to GAP SCSCP server that will pass it on to Singular.

In [1]:
import sympy

In [2]:
from sympy.polys import ring, ZZ, QQ

In [3]:
from scscp import SCSCPCLI

* Create a multivariate polynomial

In [4]:
R, x, y, z = ring("x, y, z", ZZ)

In [5]:
f = x*y*z+y**2*z+x**2*z+1

In [6]:
f

x**2*z + x*y*z + y**2*z + 1

* Presently SymPy does not implement OpenMath support for polynomials, so we will be passing their external representation instead

In [7]:
coeffs = f.coeffs()

In [8]:
coeffs

[1, 1, 1, 1]

In [9]:
mons = [ list(x) for x in f.monoms() ]

In [10]:
mons

[[2, 0, 1], [1, 1, 1], [0, 2, 1], [0, 0, 0]]

* Thus, we can have two functions to convert between the polynomials and its external representation

In [11]:
def ext_rep_poly(f):
    return [ f.coeffs(), [ list(x) for x in f.monoms() ] ]

In [12]:
from numpy import prod
def construct_poly(R,extrep):
    g = R.gens
    coeffs = extrep[0]
    mons = extrep[1]
    return sum ( [ coeffs[m]*prod([g[i]**mons[m][i] for i in range(len(g))]) for m in range(len(mons))] )

In [13]:
g = construct_poly(R,ext_rep_poly(f))

In [14]:
f == g

True

* Similar functions are defined on the GAP SCSCP server. Let's test that we can send polynomials back and forth

In [15]:
c = SCSCPCLI('localhost')

In [16]:
f == construct_poly(R, c.heads.scscp_transient_1.PingPongPoly([ ext_rep_poly(f)]))

True

In [17]:
c.quit()

* Now a small example of a Groebner basis computation

In [18]:
R, x0, x1, x2, x3 = ring("x0, x1, x2, x3", ZZ)

In [19]:
f1=x0+x1+x2+x3
f2=x0*x1+x1*x2+x0*x3+x2*x3
f3=x0*x1*x2+x0*x1*x3+x0*x2*x3+x1*x2*x3
f4=x0*x1*x2*x3-1

In [20]:
time(sympy.polys.groebnertools.groebner([f1,f2,f3,f4],R))

CPU times: user 9.42 ms, sys: 473 µs, total: 9.89 ms
Wall time: 9.64 ms


[x0 + x1 + x2 + x3,
 x1**2 + 2*x1*x3 + x3**2,
 x1*x2 - x1*x3 + x2**2*x3**4 + x2*x3 - 2*x3**2,
 x1*x3**4 - x1 + x3**5 - x3,
 x2**3*x3**2 + x2**2*x3**3 - x2 - x3,
 x2**2*x3**6 - x2**2*x3**2 - x3**4 + 1]

* The same example using remote procedure call

In [21]:
c = SCSCPCLI('localhost')

* Just another check for passing polynomials around

In [22]:
all( t == construct_poly(R, c.heads.scscp_transient_1.PingPongPoly([ ext_rep_poly(t)])) for t in [f1,f2,f3,f4] )

True

* Now actual procedure call

In [23]:
bas = c.heads.scscp_transient_1.GroebnerBasisWithSingular( [ [ ext_rep_poly(x) for x in [f1,f2,f3,f4] ] ] )

* Convert result to SymPy polynomials

In [24]:
[ construct_poly(R,t) for t in bas ]

[x0 + x1 + x2 + x3,
 x1**2 + 2*x1*x3 + x3**2,
 x1*x2**2 - x1*x3**2 + x2**2*x3 - x3**3,
 x1*x2*x3**2 - x1*x3**3 + x2**2*x3**2 + x2*x3**3 - x3**4 - 1,
 x1*x3**4 - x1 + x3**5 - x3,
 x2**3*x3**2 + x2**2*x3**3 - x2 - x3,
 x1*x2 - x1*x3 + x2**2*x3**4 + x2*x3 - 2*x3**2]

In [25]:
c.quit()

* In the next example, Singular performs much faster

In [26]:
R, x0, x1, x2, x3, x4 = ring("x0, x1, x2, x3, x4", ZZ)

In [27]:
f1=x0+x1+x2+x3+x4
f2=x0*x1+x1*x2+x2*x3+x0*x4+x3*x4
f3=x0*x1*x2+x1*x2*x3+x0*x1*x4+x0*x3*x4+x2*x3*x4
f4=x0*x1*x2*x3+x0*x1*x2*x4+x0*x1*x3*x4+x0*x2*x3*x4+x1*x2*x3*x4
f5=x0*x1*x2*x3*x4-1

* Local calculation with SymPy

In [28]:
time(sympy.polys.groebnertools.groebner([f1,f2,f3,f4,f5],R))

CPU times: user 1min 56s, sys: 915 ms, total: 1min 57s
Wall time: 1min 59s


[x0 + x1 + x2 + x3 + x4,
 275*x1**2 + 825*x1*x4 + 550*x3**6*x4 + 1650*x3**5*x4**2 + 275*x3**4*x4**3 - 550*x3**3*x4**4 + 275*x3**2 - 566*x3*x4**11 - 69003*x3*x4**6 + 69019*x3*x4 - 1467*x4**12 - 178981*x4**7 + 179073*x4**2,
 275*x1*x2 - 275*x1*x4 + 275*x2**2 + 550*x2*x4 - 330*x3**6*x4 - 1045*x3**5*x4**2 - 275*x3**4*x4**3 + 275*x3**3*x4**4 - 550*x3**2 + 334*x3*x4**11 + 40722*x3*x4**6 - 40726*x3*x4 + 867*x4**12 + 105776*x4**7 - 105873*x4**2,
 275*x1*x3 - 275*x1*x4 - 110*x3**6*x4 - 440*x3**5*x4**2 - 275*x3**4*x4**3 + 275*x3**3*x4**4 + 124*x3*x4**11 + 15092*x3*x4**6 - 15106*x3*x4 + 346*x4**12 + 42218*x4**7 - 42124*x4**2,
 55*x1*x4**5 - 55*x1 + x4**11 + 143*x4**6 - 144*x4,
 275*x2**3 + 550*x2**2*x4 - 550*x2*x4**2 + 275*x3**6*x4**2 + 550*x3**5*x4**3 - 550*x3**4*x4**4 + 550*x3**2*x4 - 232*x3*x4**12 - 28336*x3*x4**7 + 28018*x3*x4**2 - 568*x4**13 - 69289*x4**8 + 69307*x4**3,
 275*x2*x3 - 275*x2*x4 + 440*x3**6*x4 + 1210*x3**5*x4**2 - 275*x3**3*x4**4 + 275*x3**2 - 442*x3*x4**11 - 53911*x3*x4**6 + 5

* Remote calculation with Singular

In [29]:
c = SCSCPCLI('localhost')

In [30]:
all( t == construct_poly(R, c.heads.scscp_transient_1.PingPongPoly([ ext_rep_poly(t)])) for t in [f1,f2,f3,f4,f5] )

True

In [31]:
time( [ construct_poly(R,t) for t in \
       c.heads.scscp_transient_1.GroebnerBasisWithSingular( [ [ ext_rep_poly(x) for x in [f1,f2,f3,f4,f5] ] ] ) ] )

CPU times: user 5.85 s, sys: 21.9 ms, total: 5.87 s
Wall time: 6.01 s


[x0 + x1 + x2 + x3 + x4,
 x1**2 + x1*x3 + 2*x1*x4 - x2*x3 + x2*x4 + x4**2,
 x1*x2*x3 - 2*x1*x3**2 - 2*x1*x3*x4 + 3*x1*x4**2 + x2**3 + 3*x2**2*x4 - x2*x3**2 - 2*x2*x3*x4 + 3*x2*x4**2 - x3**3 - 3*x3**2*x4 - 2*x3*x4**2 + 2*x4**3,
 x1*x2**2 - x1*x2*x3 + x1*x3*x4 - x1*x4**2 + x2**2*x3 - x2**2*x4 + x2*x3*x4 - 2*x2*x4**2 + x3**2*x4 + x3*x4**2 - x4**3,
 14*x1*x2*x3*x4 - x1*x2*x4**2 - 27*x1*x3**2*x4 - 10*x1*x3*x4**2 + 24*x1*x4**3 + 6*x2**2*x3*x4 + 7*x2**2*x4**2 + 2*x2*x3**2*x4 - 9*x2*x3*x4**2 + 33*x2*x4**3 + x3**4 - 15*x3**3*x4 - 33*x3**2*x4**2 - 14*x3*x4**3 + 22*x4**4,
 32*x1*x2*x3*x4 - 24*x1*x2*x4**2 - 40*x1*x3**2*x4 - 12*x1*x3*x4**2 + 44*x1*x4**3 + 11*x2**2*x3*x4 - 3*x2**2*x4**2 + 19*x2*x3**3 + 10*x2*x3**2*x4 - 45*x2*x3*x4**2 + 32*x2*x4**3 + 5*x3**4 + x3**3*x4 - 32*x3**2*x4**2 - 32*x3*x4**3 + 34*x4**4,
 3*x1*x2*x3*x4 - 4*x1*x2*x4**2 + x1*x3**3 - 2*x1*x3**2*x4 - 2*x1*x3*x4**2 + 4*x1*x4**3 + x2**2*x3*x4 - 2*x2**2*x4**2 + 3*x2*x3**3 + 3*x2*x3**2*x4 - 7*x2*x3*x4**2 - x2*x4**3 + x3**4 + 3*x3**3*x

In [32]:
c.quit()

* Thus, the wall time was reduced from 2 minutes to 6 seconds